In [1]:
%run 'VMC.py'

/Users/hwanda/VMC/VMC.py:654: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if q0 is 0:


# Variational Monte Carlo

## Background

### Model Hamiltonian

The goal is to study the bilayer honeycomb lattice model:
$$H=-\sum_{\langle ij\rangle,l,\sigma}(c_{il\sigma}^\dagger c_{jl\sigma}+\text{h.c.})+J\sum_{i}\mathbf{S}_{i1}\cdot\mathbf{S}_{i2},$$
where $\mathbf{S}_{il}=\frac{1}{2}c_{il\alpha}^\dagger\mathbf{\sigma}_{\alpha\beta}c_{il\beta}$. $i$ - site, $l$ - layer, $\sigma$ - spin. Expectation: strong enough inter-layer Heisenberg coupling $J$ will drive a SMG transition to gap out the Dirac fermions.

### Variational Ansatz

The variational ansatz is based on the mean-field Hamiltonian
$$H_\text{MF}[u]=-\sum_{\langle ij\rangle,l,\sigma}(c_{il\sigma}^\dagger c_{jl\sigma}+\text{h.c.})+u\sum_{i,\sigma}(c_{i1\sigma}^\dagger c_{i2\sigma}+\text{h.c.}).$$
Let $|\Psi_\text{MF}[u]\rangle$ be the ground state of $H_\text{MF}[u]$, the variational state is
$$|\Psi\rangle=\mathcal{P}[E]|\Psi_\text{MF}[u]\rangle,$$
where the projection operator suppress the inter-layer charge fluctuation
$$\mathcal{P}[E]=\mathcal{P}_\text{global}\exp\left(-\sum_i E(\mathbf{q}_i)\right).$$
The charge vector on each site may include the total charge, the valey charge and the spin:
$$\mathbf{q}_i=\sum_{l,\sigma}(1,(-)^l,(-)^\sigma) n_{il\sigma}$$
with $n_{il\sigma}=c_{il\sigma}^\dagger c_{il\sigma}$. $\mathcal{P}_\text{global}$ is the additional symmetry projection operator to ensure global symmetry charge neutrality.

### Algorithm

Let ${|x\rangle}_{x\in X}$ be a complete basis of the many-body Hilbert space (with total electric charge and total valley charge neutral). The expectation value of any observable $O$ can be written as
$$\langle O\rangle=\frac{\langle \Psi|O|\Psi\rangle}{\langle\Psi|\Psi\rangle}=\sum_{x}O(x)p(x),$$
with
$$O(x)=\frac{\langle \Psi|O|x\rangle}{\langle \Psi|x\rangle}=\sum_{x'}\frac{\langle \Psi|x'\rangle}{\langle \Psi|x\rangle}\langle x'|O|x\rangle,$$
$$p(x)=\frac{|\langle \Psi|x\rangle|^2}{\sum_{x}|\langle \Psi|x\rangle|^2}.$$

To sample from $p(x)$, following the Markov chain with the transition probability
$$p(x'|x)=\frac{p(x')}{p(x)}=\left|\frac{\langle \Psi|x'\rangle}{\langle \Psi|x\rangle}\right|^2.$$

The many-body basis $|x\rangle$ is choosen to be the eigen basis of the projection operator $\mathcal{P}[g]$, such that the amplitude ratio can be evaluated as
$$\frac{\langle \Psi|x'\rangle}{\langle \Psi|x\rangle}=\frac{\langle \Psi_\text{MF}[u]|x'\rangle}{\langle \Psi_\text{MF}[u]|x\rangle}\frac{\langle x'|\mathcal{P}[E]|x'\rangle}{\langle x|\mathcal{P}[E]|x\rangle}.$$

Objective function (minimize) (let $\theta$ be the parameter for $|{\Psi}\rangle$):
$$\langle H\rangle_\theta=\sum_{x}H_\theta(x)p_\theta(x),$$
Differetial programming:
$$\begin{split}
\partial_\theta \langle H\rangle_\theta&=\sum_{x}\partial_\theta H_\theta(x)p_\theta(x)+H_\theta(x) \partial_\theta p_\theta(x)\\
&=\sum_{x}\partial_\theta H_\theta(x)p_\theta(x)+H_\theta(x) \frac{\partial_\theta p_\theta(x)}{ p_\theta(x)} p_\theta(x)\\
&=\sum_{x}(\partial_\theta H_\theta(x)+H_\theta(x) \partial_\theta \ln p_\theta(x))p_\theta(x)
\end{split}$$

## Test

### Lattice

Deform the honeycomb lattice to a square lattice, such that $x,y=0,\cdots,L-1$.

<img src="lattice.png" width=180>

red - sublattice A $(x,y)$, blue - sublattice B $(x+\frac{1}{2},y+\frac{1}{2})$. Assume periodic boundary condition.

Example: create a honeycomb lattice size $L$

In [52]:
latt = HoneycombModel(1)

Lattice sites and their corresponding site indices

In [3]:
list(latt.sites())

[(0, 0, 0), (0, 0, 1)]

Lattice bonds (pairs of site indices) and their corresponding bound strength.

In [4]:
list(latt.bonds())

[((0, 0, 0), (0, 0, 1), 1.0)]

In [5]:
latt.Ht().terms

{((0, 4),): 1.0,
 ((4, 0),): 1.0,
 ((1, 5),): 1.0,
 ((5, 1),): 1.0,
 ((2, 6),): 1.0,
 ((6, 2),): 1.0,
 ((3, 7),): 1.0,
 ((7, 3),): 1.0}

### Configuration

Fermion occupation configruation is specified by a subset of modes. The configuration class is inherated from the list class, but with additional dictionary to track the index of each mode in the list. Modes should not duplicate.

In [6]:
x = Configuration([1,3,7,4,2])
x

[1, 3, 7, 4, 2]

Check if a mode is contained in the configuration. $O(1)$ complexity.

In [7]:
3 in x, 5 in x

(True, False)

Configuration is treated as a list and can be used to index Torch tensors. 

In [8]:
torch.rand(8,4)[x]

tensor([[0.4082, 0.3264, 0.3156, 0.4925],
        [0.3501, 0.9738, 0.9768, 0.7121],
        [0.6714, 0.2511, 0.9985, 0.2498],
        [0.6490, 0.6743, 0.5948, 0.5691],
        [0.0378, 0.6525, 0.0409, 0.2165]])

Make a independent copy, such that updating the new copy will not affect the old one.

In [9]:
x1 = x.clone()
x1[0] = 5
x, x1

([1, 3, 7, 4, 2], [5, 3, 7, 4, 2])

Mode replacement can be implemented by `.replace(source, target)`.

In [10]:
x1.replace(2,8)

[5, 3, 7, 4, 8]

Invalid replacemenet leads to `ConfigurationError`

In [11]:
x = Configuration([1,3,7,4,2])
x.replace(3,7)

ConfigurationError: Mode 7 can not be created in the configuration.

### Basis System

Basis system manages many-body basis and their charge assignment. Each basis state $|{x}\rangle$ is labeled by an occupation configuration $x$ (which is a subset of modes that are occupied).

Example: random sampling a configuration

In [272]:
%run 'VMC.py'

In [273]:
bs = BasisSystem(latt)
x = bs.sample()
x

[4, 5, 2, 3]

Charge vector on each site.

In [278]:
bs.qi(x)

tensor([[-2,  0],
        [ 2,  0]])

In [275]:
bs.qmsk(x)

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]], dtype=torch.float64)

In [276]:
bs.qmsk(x, q0=torch.tensor([2.,0.]))

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]], dtype=torch.float64)

### Mean-Field Model

In [31]:
latt = HoneycombModel(1)
mf = MeanfieldModel(latt)

In [32]:
mf.H

tensor([[-0., -0., -1., -0., -1., -0., -0., -0.],
        [-0., -0., -0., -1., -0., -1., -0., -0.],
        [-1., -0., -0., -0., -0., -0., -1., -0.],
        [-0., -1., -0., -0., -0., -0., -0., -1.],
        [-1., -0., -0., -0., -0., -0.,  1., -0.],
        [-0., -1., -0., -0., -0., -0., -0.,  1.],
        [-0., -0., -1., -0.,  1., -0., -0., -0.],
        [-0., -0., -0., -1., -0.,  1., -0., -0.]], grad_fn=<AddBackward0>)

In [33]:
mf.M

tensor([[-0.0000, -0.7071,  0.0000,  0.0000],
        [ 0.1838,  0.0000, -0.5500,  0.4047],
        [-0.1926, -0.5000, -0.3135, -0.3386],
        [-0.3128, -0.0000, -0.3667,  0.5174],
        [ 0.1926, -0.5000,  0.3135,  0.3386],
        [ 0.5727, -0.0000, -0.4111,  0.0549],
        [-0.2723, -0.0000, -0.4433, -0.4789],
        [-0.6262, -0.0000,  0.0314,  0.3270]], grad_fn=<CloneBackward0>)

### ProjectorModel

In [1]:
%run "VMC.py"

/Users/hwanda/VMC/VMC.py:589: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if q0 is 0:


In [308]:
pj = ProjectorModel()
pj.energy(bs.qi(x))

tensor(-4.3944, grad_fn=<AddBackward0>)

In [313]:
pj.energy(torch.tensor([[1,0]]))

tensor(-2.1972, grad_fn=<AddBackward0>)

### Variational State

Create a variational model

In [101]:
mdl = VariationalModel(1)

Get a variational state configuration

In [102]:
st = mdl.state([4,1,2,7])
st, st.det, st.qi, st.W

(VariationalState([4, 1, 2, 7]),
 tensor(-0.2500, grad_fn=<MulBackward0>),
 tensor([[0., 0.],
         [0., 0.]], dtype=torch.float64),
 tensor([[ 7.0711e-01, -3.4110e-08, -7.0711e-01,  1.0012e-08],
         [-2.9802e-08,  1.0000e+00, -2.9802e-08,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00, -1.4901e-08],
         [-8.9407e-08, -7.0711e-01, -8.9407e-08,  7.0711e-01],
         [ 1.0000e+00,  0.0000e+00,  2.9802e-08, -2.9802e-08],
         [-6.3330e-08,  7.0711e-01, -6.3330e-08,  7.0711e-01],
         [ 7.0711e-01,  2.9802e-08,  7.0711e-01,  5.9605e-08],
         [-2.9802e-08,  0.0000e+00, -2.9802e-08,  1.0000e+00]],
        grad_fn=<MmBackward0>))

Forward the state configuration by a sequence of instructions

In [103]:
st1 = st.forward([(1,3),(2,0)])
st1, st1.det, st1.qi, st1.W

(VariationalState([4, 3, 0, 7]),
 tensor(-0.1250, grad_fn=<MulBackward0>),
 tensor([[0., 0.],
         [0., 0.]], dtype=torch.float64),
 tensor([[ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [-3.1249e-07, -1.4142e+00,  2.2096e-07,  1.0000e+00],
         [ 1.0000e+00,  6.8221e-08, -1.4142e+00, -4.8982e-08],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 1.0000e+00,  2.0331e-15, -4.2147e-08, -2.9802e-08],
         [-3.0547e-07, -1.0000e+00,  2.1600e-07,  1.4142e+00],
         [ 1.4142e+00,  6.0927e-09, -1.0000e+00,  6.5308e-08],
         [-5.9605e-08, -2.0331e-15,  4.2147e-08,  1.0000e+00]],
        grad_fn=<CopySlices>))

Backward the state configuration

In [104]:
st2 = st1.backward([(1,3),(2,0)])
st2, st2.det, st2.qi, st2.W

(VariationalState([4, 1, 2, 7]),
 tensor(-0.2500, grad_fn=<MulBackward0>),
 tensor([[0., 0.],
         [0., 0.]], dtype=torch.float64),
 tensor([[ 7.0711e-01, -3.4110e-08, -7.0711e-01, -5.2517e-10],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [-1.1048e-07, -7.0711e-01, -1.1048e-07,  7.0711e-01],
         [ 1.0000e+00,  0.0000e+00,  2.9802e-08, -2.9802e-08],
         [-4.2257e-08,  7.0711e-01, -4.2257e-08,  7.0711e-01],
         [ 7.0711e-01,  2.9802e-08,  7.0711e-01,  7.0141e-08],
         [-2.9802e-08,  0.0000e+00, -2.9802e-08,  1.0000e+00]],
        grad_fn=<CopySlices>))

Invalid instruction leads to error.

In [100]:
st.forward([(3,4)])

ConfigurationError: Mode 3 can not be annihilated in the configuration.

### Variational Model

Create a variational model

In [2]:
mdl = VariationalModel(1)
st = mdl.state([0,1,2,3])

In [3]:
st1, logq = mdl.propose2(st)
st1, logq, st1.logprob - st.logprob

(VariationalState([0, 5, 2, 3]),
 tensor(1.0986, dtype=torch.float64),
 tensor(0.6931, dtype=torch.float64, grad_fn=<SubBackward0>))

In [5]:
configs = [sorted(st.config)]
for _ in range(1000):
    st, _ = mdl.MCstep(st)
    configs.append(sorted(st.config))
configs, counts = torch.unique(torch.tensor(configs), return_counts=True, dim=0)
ps = torch.stack([mdl.state(config.tolist()).logprob.exp() for config in configs])
ps = ps/ ps.sum()
for config, count, p in zip(configs, counts, ps):
    print('{} {:6d} {:6.3f}'.format(config.tolist(), count.item(), p.item()))

[0, 1, 2, 3]     25  0.027
[0, 1, 2, 7]     48  0.054
[0, 1, 3, 6]     56  0.054
[0, 1, 6, 7]     96  0.108
[0, 2, 3, 5]     66  0.054
[0, 2, 5, 7]     21  0.027
[0, 3, 5, 6]    107  0.108
[0, 5, 6, 7]     46  0.054
[1, 2, 3, 4]     49  0.054
[1, 2, 4, 7]     88  0.108
[1, 2, 5, 6]     30  0.027
[1, 3, 4, 6]     38  0.027
[1, 4, 6, 7]     59  0.054
[2, 3, 4, 5]    132  0.108
[2, 4, 5, 7]     43  0.054
[3, 4, 5, 6]     66  0.054
[4, 5, 6, 7]     31  0.027


In [6]:
%run 'VMC.py'

/Users/hwanda/VMC/VMC.py:589: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if q0 is 0:


In [320]:
mdl = VariationalModel(1)
mdl.meanfield.u = torch.nn.Parameter(torch.tensor(0.))
ps = torch.stack([mdl.state(config.tolist()).logprob.exp() for config in configs])
ps, ps.sum()

(tensor([0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0000, 0.0008, 0.0008,
         0.0008, 0.0008, 0.0000, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008],
        dtype=torch.float64, grad_fn=<StackBackward0>),
 tensor(0.0123, dtype=torch.float64, grad_fn=<SumBackward0>))

Rejection rate is low.

mdl._rejects/ mdl._step

### Operator

In [40]:
mdl = VariationalModel(1)
st = mdl.state([0,1,2,3])
op = - mdl.lattice.Ht + mdl.lattice.HJ
op.terms

{((0, 0), (2, 2)): 0.25,
 ((0, 0), (3, 3)): -0.25,
 ((1, 1), (2, 2)): -0.25,
 ((1, 1), (3, 3)): 0.25,
 ((0, 2), (3, 1)): -0.5,
 ((2, 0), (1, 3)): -0.5,
 ((4, 6), (7, 5)): -0.5,
 ((6, 4), (5, 7)): -0.5,
 ((4, 4), (6, 6)): 0.25,
 ((4, 4), (7, 7)): -0.25,
 ((5, 5), (6, 6)): -0.25,
 ((5, 5), (7, 7)): 0.25,
 ((0, 4),): -1.0,
 ((4, 0),): -1.0,
 ((1, 5),): -1.0,
 ((5, 1),): -1.0,
 ((2, 6),): -1.0,
 ((6, 2),): -1.0,
 ((3, 7),): -1.0,
 ((7, 3),): -1.0}

Evaulate $O(x)$.

In [13]:
op.evaluate(st)

tensor(-5.6569, dtype=torch.float64, grad_fn=<AddBackward0>)

### VMC

In [1]:
%run 'VMC.py'

/Users/hwanda/VMC/VMC.py:665: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if q0 is 0:


In [20]:
mdl = VariationalModel(1)
H = - mdl.lattice.Ht + 5*mdl.lattice.HJ
state = mdl.MCrun(steps=10).reset()

In [21]:
optimizer = torch.optim.Adam(mdl.parameters(), lr=0.01)

In [22]:
for iteration in range(10):
    Hval, state = mdl.MCrun(H=H, state=state, steps=100)
    loss = Hval #- Hval.detach()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    mdl.reset()
    state.reset()
    print('{} {}'.format(loss.item()/5, Hval.item()/5))

-1.1164499269370554 -1.1164499269370554
-0.9709138219583201 -0.9709138219583201
-1.095825029053172 -1.095825029053172
-0.958490807643851 -0.958490807643851
-0.9325108321900876 -0.9325108321900876
-1.1173666922735825 -1.1173666922735825
-0.9744966948882666 -0.9744966948882666
-0.9829344810533547 -0.9829344810533547
-0.9040803966375979 -0.9040803966375979
-0.9122904338789626 -0.9122904338789626


In [5]:
configs = [[0, 1, 2, 3],
 [0, 1, 2, 7],
 [0, 1, 3, 6],
 [0, 1, 6, 7],
 [0, 2, 3, 5],
 [0, 2, 5, 7],
 [0, 3, 4, 7],
 [0, 3, 5, 6],
 [0, 5, 6, 7],
 [1, 2, 3, 4],
 [1, 2, 4, 7],
 [1, 2, 5, 6],
 [1, 3, 4, 6],
 [1, 4, 6, 7],
 [2, 3, 4, 5],
 [2, 4, 5, 7],
 [3, 4, 5, 6],
 [4, 5, 6, 7]]
ps = []
Hval = 0
for config in configs:
    ps.append(mdl.state(config).logprob.exp().item())
ampls = [(float(p)/sum(ps)) ** 0.5 for p in ps]
for config, ampl in zip(configs, ampls):
    print('{} {}'.format(config, ampl))
    Hval += H.evaluate(mdl.state(config)) * ampl**2
Hval

[0, 1, 2, 3] 0.1969028838557125
[0, 1, 2, 7] 0.22103719317402273
[0, 1, 3, 6] 0.22103719317402273
[0, 1, 6, 7] 0.29306304792012655
[0, 2, 3, 5] 0.22103719317402273
[0, 2, 5, 7] 0.16128398269895833
[0, 3, 4, 7] 0.1608896898826247
[0, 3, 5, 6] 0.35779257373833717
[0, 5, 6, 7] 0.22103719317402284
[1, 2, 3, 4] 0.22103719317402265
[1, 2, 4, 7] 0.35779257373833717
[1, 2, 5, 6] 0.1608896898826247
[1, 3, 4, 6] 0.16128398269895847
[1, 4, 6, 7] 0.22103719317402273
[2, 3, 4, 5] 0.29306304792012644
[2, 4, 5, 7] 0.22103719317402265
[3, 4, 5, 6] 0.22103719317402273
[4, 5, 6, 7] 0.19690288385571245


tensor(-5.3593, dtype=torch.float64, grad_fn=<AddBackward0>)

In [7]:
mdl.meanfield.u, mdl.projector.weight

(Parameter containing:
 tensor(2.2140, requires_grad=True),
 Parameter containing:
 tensor([[-0.0573,  0.0941, -0.0437],
         [-0.1131,  0.0941,  0.0722]], requires_grad=True))

In [8]:
state, state.qi, state.energy

(VariationalState([4, 5, 6, 3]),
 tensor([[-1, -1],
         [ 1,  1]]),
 tensor(4.0621, grad_fn=<AddBackward0>))

### Autograd

In [177]:
class EigVecsH(torch.autograd.Function):
    @staticmethod
    def forward(ctx, A, UPLO, tol):
        L, Q = torch.linalg.eigh(A, UPLO=UPLO)
        ctx.save_for_backward(Q)
        ctx.L = L
        ctx.requires_grad_A = A.requires_grad
        ctx.tol = tol
        return Q
    
    @staticmethod
    def backward(ctx, grad_Q):
        (Q,) = ctx.saved_tensors
        if ctx.requires_grad_A:
            QH = Q.T.conj()
            G = QH.mm(grad_Q)
            G = (G - G.T.conj())/2
            dL = ctx.L.unsqueeze(0) - ctx.L.unsqueeze(1)
            F = dL / (dL**2 + ctx.tol**2)
            grad_A = Q.mm(G * F).mm(QH)
            return grad_A, None, None
        else:
            return None, None, None
        
def eigvecsh(A, UPLO='L', tol=1.e-10):
    return EigVecsH.apply(A, UPLO, tol)

In [178]:
X = torch.tensor([[0.,1.],[1.,0.]])
Z = torch.tensor([[1.,0.],[0.,-1.]])
I = torch.tensor([[1.,0.],[0.,1.]])
X = torch.kron(X, I)
Z = torch.kron(Z, I)

In [179]:
a = torch.tensor(3.).requires_grad_() 
b = torch.tensor(4.).requires_grad_() 
A = - a*X - b*Z
print('A: \n',A)
U = eigvecsh(A, tol=1.e-5)
print('U: \n',U)
M = U[:,:(A.shape[-1]//2)]
print('M: \n',M)
x = M.T.conj().mm(X).mm(M).trace()
z = M.T.conj().mm(Z).mm(M).trace()
print('x: \n', x, '\nz: \n', z)
x.backward()
a.grad, b.grad

A: 
 tensor([[-4., -0., -3., -0.],
        [-0., -4., -0., -3.],
        [-3., -0.,  4.,  0.],
        [-0., -3.,  0.,  4.]], grad_fn=<SubBackward0>)
U: 
 tensor([[-0.9487, -0.0000,  0.0000, -0.3162],
        [ 0.0000, -0.9487, -0.3162,  0.0000],
        [-0.3162,  0.0000,  0.0000,  0.9487],
        [-0.0000, -0.3162,  0.9487,  0.0000]], grad_fn=<EigVecsHBackward>)
M: 
 tensor([[-0.9487, -0.0000],
        [ 0.0000, -0.9487],
        [-0.3162,  0.0000],
        [-0.0000, -0.3162]], grad_fn=<SliceBackward0>)
x: 
 tensor(1.2000, grad_fn=<TraceBackward0>) 
z: 
 tensor(1.6000, grad_fn=<TraceBackward0>)


(tensor(0.2560), tensor(-0.1920))

In [32]:
class UpdateW(torch.autograd.Function):
    @staticmethod
    def forward(ctx, W, mode_tgt, idx):
        ratio = W[mode_tgt, idx].clone()
        Wcol = W[:, idx].clone()
        Wrow = W[mode_tgt, :].clone()
        Wrow[idx] -= 1.
        ctx.need_grad = W.requires_grad
        if ctx.need_grad:
            ctx.mode_tgt, ctx.idx, ctx.ratio, ctx.Wcol, ctx.Wrow = mode_tgt, idx, ratio, Wcol, Wrow
        return W - Wrow.unsqueeze(0) * Wcol.unsqueeze(1) / ratio
        
    @staticmethod
    def backward(ctx, grad_W):
        grad_Wout = None
        if ctx.need_grad:
            grad_Wcol = grad_W @ ctx.Wrow / ctx.ratio
            grad_Wrow = ctx.Wcol @ grad_W / ctx.ratio
            grad_ratio = grad_Wrow @ ctx.Wrow / ctx.ratio
            grad_Wout = grad_W.clone()
            grad_Wout[:, ctx.idx] -= grad_Wcol
            grad_Wout[ctx.mode_tgt, :] -= grad_Wrow
            grad_Wout[ctx.mode_tgt, ctx.idx] += grad_ratio
        return grad_Wout, None, None
    
updateW = UpdateW.apply

In [72]:
W0 = VariationalModel(1).state([0,1,2,7]).W.detach().requires_grad_()
W0

tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 4.7706e-18,  1.0000e+00,  5.7582e-18,  0.0000e+00],
        [ 0.0000e+00, -5.2155e-18,  1.0000e+00,  0.0000e+00],
        [ 6.4585e-17, -7.0711e-01,  8.1220e-17,  7.0711e-01],
        [ 1.4142e+00,  3.0709e-16,  1.0000e+00,  0.0000e+00],
        [ 7.4192e-17,  7.0711e-01,  9.9191e-17,  7.0711e-01],
        [ 1.0000e+00,  5.2736e-16,  1.4142e+00,  3.9252e-17],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
       dtype=torch.float64, requires_grad=True)

In [73]:
W1 = updateW(W0, 4, 2)
W1

tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-3.3727e-18,  1.0000e+00,  5.7582e-18,  0.0000e+00],
        [-1.4142e+00, -3.1231e-16,  1.0000e+00,  0.0000e+00],
        [-5.0277e-17, -7.0711e-01,  8.1220e-17,  7.0711e-01],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
        [-6.6085e-17,  7.0711e-01,  9.9191e-17,  7.0711e-01],
        [-1.0000e+00,  9.3065e-17,  1.4142e+00,  3.9252e-17],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
       dtype=torch.float64, grad_fn=<UpdateWBackward>)

In [74]:
for i in range(8):
    for j in range(4):
        def func(W):
            return updateW(W, 4, 2)[i,j]
        print(torch.autograd.gradcheck(func, W0))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [294]:
idx = qi.abs()
idx

tensor([[2, 0],
        [2, 0]])

In [295]:
weight = torch.tensor([[0.,1.,2.],[0.,0.,0.]])
mu = torch.nn.functional.log_softmax(weight,-1)
mu

tensor([[-2.4076, -1.4076, -0.4076],
        [-1.0986, -1.0986, -1.0986]])

In [301]:
mu[0,idx[:,0]].sum() + mu[1,idx[:,1]].sum()

(tensor(-0.8152), tensor(-2.1972))

In [302]:
sum(mu[i,idx[:,i]].sum() for i in range(2))

tensor(-3.0124)

# Train two-site model

In [1]:
%run 'VMC.py'

/Users/hwanda/VMC/VMC.py:651: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if q0 is 0:


### exact optimization

In [4]:
configs = [[0, 1, 2, 3],
 [0, 1, 2, 7],
 [0, 1, 3, 6],
 [0, 1, 6, 7],
 [0, 2, 3, 5],
 [0, 2, 5, 7],
 [0, 3, 4, 7],
 [0, 3, 5, 6],
 [0, 5, 6, 7],
 [1, 2, 3, 4],
 [1, 2, 4, 7],
 [1, 2, 5, 6],
 [1, 3, 4, 6],
 [1, 4, 6, 7],
 [2, 3, 4, 5],
 [2, 4, 5, 7],
 [3, 4, 5, 6],
 [4, 5, 6, 7]]

mdl = VariationalModel(1)
H = - 0.2 * mdl.lattice.Ht + mdl.lattice.HJ
optimizer = torch.optim.Adam(mdl.parameters(), lr = 0.01)

for _ in range(3000):
    Hval ,configs = mdl.exct_run(H=H,configs=configs)
    loss = Hval
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    mdl.reset()
    if _ % 99 == 0:
        print(loss.item())

-1.0303394841464057
-1.5102617270809378
-1.545729838645621
-1.5549012616987408
-1.5587296836661937
-1.5606989130978626
-1.5618387217340817
-1.5625504929276506
-1.5630193402339474
-1.5633406213842143
-1.5635674679493812
-1.5637313545232605
-1.5638518367941572
-1.5639415652776505
-1.564009023779518
-1.5640600865923464
-1.5640988872628436
-1.5641284250801089
-1.5641509252948074
-1.5641680372866469
-1.5641810167640993
-1.564190813616637
-1.5641981790576067
-1.5642036728856605
-1.5642077257616793
-1.5642107067181874
-1.5642128670444875
-1.564214410558721
-1.5642155012317032
-1.5642162622627187
-1.564216784144059


In [5]:
test(H, mdl)

[0, 1, 2, 3] -0.03643906529819733
[0, 1, 2, 7] -0.07801323630820518
[0, 1, 3, 6] 0.07801323630820516
[0, 1, 6, 7] -0.03990479588956507
[0, 2, 3, 5] -0.07801323630820525
[0, 2, 5, 7] 0.030222788062085105
[0, 3, 4, 7] 0.4671453929326423
[0, 3, 5, 6] -0.5035844582308399
[0, 5, 6, 7] -0.07801323630820525
[1, 2, 3, 4] 0.07801323630820511
[1, 2, 4, 7] -0.5035844582308392
[1, 2, 5, 6] 0.46714539293264207
[1, 3, 4, 6] 0.030222788062085077
[1, 4, 6, 7] 0.07801323630820518
[2, 3, 4, 5] -0.039904795889565074
[2, 4, 5, 7] -0.07801323630820518
[3, 4, 5, 6] 0.07801323630820522
[4, 5, 6, 7] -0.03643906529819733
energy = -1.564216908158246 normalization factor = 0.01047977322417201 fidelity = 0.9877903667806816


### "agent free" policy gradient optimization

#### return plcy=0 for rejection

In [2]:
mdl = VariationalModel(1)
H = -0.2* mdl.lattice.Ht + mdl.lattice.HJ
state = mdl.MCrun(steps=10).reset()
optimizer = torch.optim.Adam(mdl.parameters(), lr=0.01)

for iteration in range(500):
    rwd, Hval, state = mdl.MCrun_pg(H=H, state=state, steps=100)
    loss = rwd
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if iteration % 99 ==0:
        print('reward =', - loss.item(),'enery =', Hval.item(), 'rejection rate =',mdl._rejects/mdl._step)
    mdl.reset()
    state.reset()

reward = 0.42676827757349756 enery = -0.8930184694319778 rejection rate = 0.05454545454545454
reward = 0.6380398217149059 enery = -1.4323707713128098 rejection rate = 0.13
reward = 0.6515165289061414 enery = -1.5586111348420872 rejection rate = 0.17
reward = 0.69483742085273 enery = -1.6066518659391542 rejection rate = 0.15
reward = 0.6765860597664118 enery = -1.55037837165495 rejection rate = 0.14
reward = 0.7557252879235208 enery = -1.6091763237857473 rejection rate = 0.07


In [3]:
test(H, mdl)

[0, 1, 2, 3] -0.09546592161198394
[0, 1, 2, 7] -0.10894306799040362
[0, 1, 3, 6] 0.10894306799040362
[0, 1, 6, 7] -0.0764293610349696
[0, 2, 3, 5] -0.10894306799040358
[0, 2, 5, 7] 0.041784150541509676
[0, 3, 4, 7] 0.41663093529211453
[0, 3, 5, 6] -0.5120968569040982
[0, 5, 6, 7] -0.10894306799040358
[1, 2, 3, 4] 0.10894306799040362
[1, 2, 4, 7] -0.5120968569040983
[1, 2, 5, 6] 0.41663093529211437
[1, 3, 4, 6] 0.041784150541509676
[1, 4, 6, 7] 0.10894306799040358
[2, 3, 4, 5] -0.07642936103496958
[2, 4, 5, 7] -0.10894306799040358
[3, 4, 5, 6] 0.10894306799040358
[4, 5, 6, 7] -0.0954659216119839
energy = -1.5405187276607923 normalization factor = 0.014774703008139357 fidelity = 0.9716204785817014


#### return plcy=-p(x')/p(x)+p(x') for rejection

In [2]:
mdl = VariationalModel(1)
H = -0.2* mdl.lattice.Ht + mdl.lattice.HJ
state = mdl.MCrun(steps=10).reset()
optimizer = torch.optim.Adam(mdl.parameters(), lr=0.01)

for iteration in range(500):
    rwd, Hval, state = mdl.MCrun_pg(H=H, state=state, steps=100)
    loss = rwd
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if iteration % 99 ==0:
        print('reward =', - loss.item(),'enery =', Hval.item(), 'rejection rate =',mdl._rejects/mdl._step)
    mdl.reset()
    state.reset()

reward = 0.33505532914754177 enery = -0.8288763338082471 rejection rate = 0.1
reward = 0.3252004631121784 enery = -1.017348096860554 rejection rate = 0.38
reward = 0.5463910528339057 enery = -1.398356522333816 rejection rate = 0.2
reward = 0.5533694198627749 enery = -1.4362776634059762 rejection rate = 0.22
reward = 0.40250878012142555 enery = -0.9701075496788516 rejection rate = 0.44
reward = 0.5005185707466694 enery = -1.324474896834815 rejection rate = 0.24


In [3]:
test(H, mdl)

[0, 1, 2, 3] -0.14556121792538101
[0, 1, 2, 7] -0.03690893583172228
[0, 1, 3, 6] 0.03690893583172228
[0, 1, 6, 7] -0.0115016628302367
[0, 2, 3, 5] -0.036908935831722264
[0, 2, 5, 7] 0.047460729683313344
[0, 3, 4, 7] 0.40700268090225783
[0, 3, 5, 6] -0.5525638988276386
[0, 5, 6, 7] -0.03690893583172228
[1, 2, 3, 4] 0.036908935831722264
[1, 2, 4, 7] -0.5525638988276388
[1, 2, 5, 6] 0.4070026809022577
[1, 3, 4, 6] 0.047460729683313344
[1, 4, 6, 7] 0.03690893583172228
[2, 3, 4, 5] -0.011501662830236693
[2, 4, 5, 7] -0.03690893583172228
[3, 4, 5, 6] 0.036908935831722264
[4, 5, 6, 7] -0.1455612179253811
energy = -1.457724020349126 normalization factor = 0.033635787728758415 fidelity = 0.968492190108913


#### grad H.evaluate()

In [2]:
mdl = VariationalModel(1)
H = -0.2* mdl.lattice.Ht + mdl.lattice.HJ
state = mdl.MCrun(steps=10).reset()
optimizer = torch.optim.Adam(mdl.parameters(), lr=0.01)

for iteration in range(3000):
    rwd, Hval, state = mdl.MCrun_pg(H=H, state=state, steps=100)
    loss = rwd
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if iteration % 99 ==0:
        print('reward =', - loss.item(),'enery=', Hval.item(), 'rejection rate =',mdl._rejects/mdl._step)
    mdl.reset()
    state.reset()

reward = 0.5031310481583539 enery= -1.015592062560785 rejection rate = 0.045454545454545456
reward = 0.3410032525492095 enery= -0.8679898432978174 rejection rate = 0.28
reward = 0.5975525133949598 enery= -1.333036661698833 rejection rate = 0.17
reward = 0.23273667078476007 enery= -0.4864008781593779 rejection rate = 0.49
reward = -0.06439309001222317 enery= 0.4440902759463659 rejection rate = 0.71
reward = -0.0739680909603749 enery= 0.4772134900669352 rejection rate = 0.69
reward = -0.1017793209898603 enery= 0.49648449263346406 rejection rate = 0.59
reward = -0.07445820415632083 enery= 0.49638802770880547 rejection rate = 0.7
reward = -0.06946998145447696 enery= 0.4962141532462639 rejection rate = 0.72
reward = -0.08184154702141253 enery= 0.4960093758873475 rejection rate = 0.67
reward = -0.05949179458186245 enery= 0.4957649548488536 rejection rate = 0.76
reward = -0.09434194508332251 enery= 0.4965365530701172 rejection rate = 0.62
reward = -0.08688432118468362 enery= 0.496481835341050

In [3]:
test(H, mdl)

[0, 1, 2, 3] -0.010493961680744426
[0, 1, 2, 7] -0.0011600090590666633
[0, 1, 3, 6] 0.0011600090590666635
[0, 1, 6, 7] -0.0008322284617255295
[0, 2, 3, 5] -0.001160009059066664
[0, 2, 5, 7] 0.7068418414644718
[0, 3, 4, 7] 0.004836451591053823
[0, 3, 5, 6] -0.015330413271798253
[0, 5, 6, 7] -0.0011600090590666635
[1, 2, 3, 4] 0.001160009059066663
[1, 2, 4, 7] -0.015330413271798246
[1, 2, 5, 6] 0.004836451591053823
[1, 3, 4, 6] 0.7068418414644715
[1, 4, 6, 7] 0.001160009059066663
[2, 3, 4, 5] -0.0008322284617255297
[2, 4, 5, 7] -0.0011600090590666635
[3, 4, 5, 6] 0.0011600090590666635
[4, 5, 6, 7] -0.010493961680744427
energy = 0.49634762930828585 normalization factor = 0.04386413685930455


#### bias optimization

In [2]:
mdl = VariationalModel(1)
H = -0.2* mdl.lattice.Ht + mdl.lattice.HJ
state = mdl.MCrun(steps=10).reset()
optimizer = torch.optim.Adam(mdl.parameters(), lr=0.01)

for iteration in range(3000):
    rwd, Hval, state = mdl.MCrun_pg(H=H, state=state, steps=100)
    loss = rwd
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if iteration % 99 ==0:
        print('reward =', - loss.item(),'enery=', Hval.item(), 'rejection rate =',mdl._rejects/mdl._step)
    mdl.reset()
    state.reset()

reward = 0.0004090446690048303 enery= -0.8719595949289324 rejection rate = 0.1
reward = 0.0268680122309219 enery= -1.1337494141996383 rejection rate = 0.34
reward = 0.06891436488064462 enery= -1.5359247869445527 rejection rate = 0.05
reward = 0.07417002268440549 enery= -1.4672237869113252 rejection rate = 0.11
reward = 0.08050424545559491 enery= -1.5683835813596678 rejection rate = 0.04
reward = 0.0808905209285094 enery= -1.552574083469712 rejection rate = 0.06
reward = 0.07837292595576127 enery= -1.4825536424158392 rejection rate = 0.12
reward = 0.0856239983271111 enery= -1.5352223332295742 rejection rate = 0.03
reward = 0.08607139541738514 enery= -1.474024983032503 rejection rate = 0.05
reward = 0.08374751235028116 enery= -1.4735711558701565 rejection rate = 0.08
reward = 0.0847007653603649 enery= -1.5138393011572666 rejection rate = 0.06
reward = 0.08854459708325438 enery= -1.4858732907351577 rejection rate = 0.03
reward = 0.08705781573502591 enery= -1.5117136609961435 rejection rat

In [3]:
test(H, mdl) # print H.eval(st)

[0, 1, 2, 3] 0.005606794366289215
[0, 1, 2, 7] 1.2888889483116284e-10
[0, 1, 3, 6] -1.288888948311629e-10
[0, 1, 6, 7] 2.5190504050614362e-05
[0, 2, 3, 5] 1.288888948311629e-10
[0, 2, 5, 7] -2.7472915156009086e-07
[0, 3, 4, 7] -0.49717302483631626
[0, 3, 5, 6] 0.5027798192026056
[0, 5, 6, 7] 1.288888948311629e-10
[1, 2, 3, 4] -1.288888948311629e-10
[1, 2, 4, 7] 0.5027798192026056
[1, 2, 5, 6] -0.4971730248363165
[1, 3, 4, 6] -2.7472915156009086e-07
[1, 4, 6, 7] -1.288888948311629e-10
[2, 3, 4, 5] 2.5190504050614362e-05
[2, 4, 5, 7] 1.288888948311629e-10
[3, 4, 5, 6] -1.288888948311629e-10
[4, 5, 6, 7] 0.00560679436628922
energy = -1.4998428175903764 normalization factor = 0.24714552617589042


### Stochastic Reconfiguration

#### 100 steps, 500 taining loop

In [6]:
mdl = VariationalModel(1)
H = -0.2* mdl.lattice.Ht + mdl.lattice.HJ
state = mdl.MCrun(steps=10).reset()
optimizer = torch.optim.Adam(mdl.parameters(), lr=0.01)
start = time.time()
for iteration in range(500):
    obj, Hval, state = mdl.MCrun_sr(H=H, state=state, steps=100)
    loss = obj #- Hval.detach()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if iteration%49 ==0:
        print('obj =', loss.item(), 'energy =', Hval.item(), 'rejection rate =',mdl._rejects/mdl._step)
    mdl.reset()
    state.reset()
end = time.time()
print('seconds =',end - start)
test(H, mdl)

obj = 0.037870017183530766 energy = -1.151449926937054 rejection rate = 0.13636363636363635
obj = -0.11800194885780706 energy = -1.2780598525611575 rejection rate = 0.16
obj = 0.018226566768772264 energy = -1.8613020170980936 rejection rate = 0.06
obj = -0.13839298192271743 energy = -1.4797356454967079 rejection rate = 0.17
obj = 0.025310907323700286 energy = -1.5784050980209838 rejection rate = 0.08
obj = -0.08096803498495703 energy = -1.489909874651052 rejection rate = 0.24
obj = 0.011871325093821343 energy = -1.5728619111253836 rejection rate = 0.11
obj = 0.00828138441755178 energy = -1.5244338895534453 rejection rate = 0.11
obj = 0.015777519103283632 energy = -1.486390753372354 rejection rate = 0.15
obj = -0.13933289936303817 energy = -1.5419745292066551 rejection rate = 0.06
obj = -0.10601282341245387 energy = -1.5495272714673536 rejection rate = 0.07
seconds = 67.82742404937744
[0, 1, 2, 3] 0.0715573865757192
[0, 1, 2, 7] 0.0872030876513947
[0, 1, 3, 6] -0.08720308765139474
[0, 1

In [7]:
mdl = VariationalModel(1)
H = -0.2* mdl.lattice.Ht + mdl.lattice.HJ
state = mdl.MCrun(steps=10).reset()
optimizer = torch.optim.Adam(mdl.parameters(), lr=0.01)
start = time.time()
for iteration in range(500):
    obj, Hval, state = mdl.MCrun_pg(H=H, state=state, steps=100)
    loss = obj
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if iteration%49 ==0:
        print('obj =', loss.item(), 'energy =', Hval.item(), 'rejection rate =',mdl._rejects/mdl._step)
    mdl.reset()
    state.reset()
end = time.time()
print('seconds',end - start)
test(H, mdl)

obj = -0.4287709134747836 energy = -0.8877636354360388 rejection rate = 0.045454545454545456
obj = -0.5073003989518441 energy = -1.1100572957075778 rejection rate = 0.11
obj = -0.6354606826205813 energy = -1.4714564705722335 rejection rate = 0.15
obj = -0.825548089754977 energy = -1.7755130351793047 rejection rate = 0.09
obj = -0.6718671134492595 energy = -1.5460483193373848 rejection rate = 0.13
obj = -0.6653442218322294 energy = -1.4950409290084483 rejection rate = 0.16
obj = -0.7468078599812356 energy = -1.6264000776630925 rejection rate = 0.08
obj = -0.7366311200984712 energy = -1.6058596363902302 rejection rate = 0.09
obj = -0.66208505443296 energy = -1.5334194969951387 rejection rate = 0.13
obj = -0.6980662579034632 energy = -1.5584504021013226 rejection rate = 0.12
obj = -0.7821197047135432 energy = -1.693645932206653 rejection rate = 0.08
seconds 69.633780002594
[0, 1, 2, 3] 0.08748984891652724
[0, 1, 2, 7] 0.09980716246993103
[0, 1, 3, 6] -0.09980716246993108
[0, 1, 6, 7] 0.06

#### 1000 steps, 100 training loop

In [9]:
mdl = VariationalModel(1)
H = -0.2* mdl.lattice.Ht + mdl.lattice.HJ
state = mdl.MCrun(steps=10).reset()
optimizer = torch.optim.Adam(mdl.parameters(), lr=0.01)
start = time.time()
for iteration in range(100):
    obj, Hval, state = mdl.MCrun_sr(H=H, state=state, steps=1000)
    loss = obj #- Hval.detach()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if iteration%9 ==0:
        print('obj =', loss.item(), 'energy =', Hval.item(), 'rejection rate =',mdl._rejects/mdl._step)
    mdl.reset()
    state.reset()
end = time.time()
print('seconds =',end - start)
test(H, mdl)

obj = 0.021748832306537666 energy = -0.9733709514213379 rejection rate = 0.0891089108910891
obj = 0.029406418148428225 energy = -1.1773747314037954 rejection rate = 0.071
obj = -0.04241790061398064 energy = -1.1621397601912415 rejection rate = 0.132
obj = -0.0938374932172493 energy = -1.301332445139461 rejection rate = 0.159
obj = -0.10684488523296244 energy = -1.4341319298507544 rejection rate = 0.136
obj = -0.08448558370214822 energy = -1.4685987409746897 rejection rate = 0.16
obj = -0.1128438075764028 energy = -1.3818987714798239 rejection rate = 0.15
obj = -0.0977876448765299 energy = -1.579470547310811 rejection rate = 0.12
obj = -0.05869450184129363 energy = -1.4454148134581848 rejection rate = 0.174
obj = -0.07953152363777397 energy = -1.5004188195624852 rejection rate = 0.168
obj = -0.04577502618158632 energy = -1.4997838743382912 rejection rate = 0.158
obj = -0.08177418755137811 energy = -1.567493787775867 rejection rate = 0.147
seconds = 137.53303980827332
[0, 1, 2, 3] 0.1249

In [10]:
mdl = VariationalModel(1)
H = -0.2* mdl.lattice.Ht + mdl.lattice.HJ
state = mdl.MCrun(steps=10).reset()
optimizer = torch.optim.Adam(mdl.parameters(), lr=0.01)
start = time.time()
for iteration in range(100):
    obj, Hval, state = mdl.MCrun_pg(H=H, state=state, steps=1000)
    loss = obj #- Hval.detach()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if iteration%9 ==0:
        print('obj =', loss.item(), 'energy =', Hval.item(), 'rejection rate =',mdl._rejects/mdl._step)
    mdl.reset()
    state.reset()
end = time.time()
print('seconds',end - start)
test(H, mdl)

obj = -0.49402935264914716 energy = -1.0381724666115029 rejection rate = 0.07227722772277227
obj = -0.5002273894210174 energy = -1.1107785447455116 rejection rate = 0.102
obj = -0.5548608119185343 energy = -1.2077565096494354 rejection rate = 0.096
obj = -0.5812055456346633 energy = -1.2886294342914864 rejection rate = 0.128
obj = -0.6310886865182672 energy = -1.4160953446227207 rejection rate = 0.137
obj = -0.600655476347467 energy = -1.3824044190062401 rejection rate = 0.156
obj = -0.5825110005010444 energy = -1.34342205961877 rejection rate = 0.158
obj = -0.6090960979479609 energy = -1.3871834991195424 rejection rate = 0.144
obj = -0.6217290052757238 energy = -1.429735243456281 rejection rate = 0.155
obj = -0.651942891862603 energy = -1.4560063605496782 rejection rate = 0.124
obj = -0.6127523036802274 energy = -1.4007818322352903 rejection rate = 0.156
obj = -0.6811740305445824 energy = -1.5249823486843357 rejection rate = 0.126
seconds 142.13578987121582
[0, 1, 2, 3] 0.136528246772

#### biased without '-Hval'

In [2]:
mdl = VariationalModel(1)
H = -0.2* mdl.lattice.Ht + mdl.lattice.HJ
state = mdl.MCrun(steps=10).reset()
optimizer = torch.optim.Adam(mdl.parameters(), lr=0.01)
start = time.time()
for iteration in range(500):
    obj, Hval, state = mdl.MCrun_sr(H=H, state=state, steps=100)
    loss = obj #- Hval.detach()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if iteration%49 ==0:
        print('obj =', loss.item(), 'energy =', Hval.item(), 'rejection rate =',mdl._rejects/mdl._step)
    mdl.reset()
    state.reset()
end = time.time()
print('seconds =',end - start)
test(H, mdl)

obj = 8.195200874574986 energy = -1.0431656556895925 rejection rate = 0.1
obj = 7.04801992627338 energy = -1.2212958364148483 rejection rate = 0.28
obj = 6.416489911080817 energy = -1.4401023263555737 rejection rate = 0.26
obj = 6.549591906636346 energy = -1.6631599869082538 rejection rate = 0.11
obj = 4.081246467802258 energy = -1.2803291869179814 rejection rate = 0.22
obj = 4.718536866201032 energy = -1.4589237700174118 rejection rate = 0.11
obj = 4.683637082053294 energy = -1.4838090382936886 rejection rate = 0.15
obj = 4.726504461681505 energy = -1.516078546627182 rejection rate = 0.12
obj = 3.425439461571637 energy = -1.1278420549780206 rejection rate = 0.33
obj = 4.985211095549612 energy = -1.6012916365774807 rejection rate = 0.09
obj = 4.449459664993982 energy = -1.4927941902719428 rejection rate = 0.09
seconds = 69.96335101127625
[0, 1, 2, 3] 0.05855015221772714
[0, 1, 2, 7] 2.1742918942620318e-05
[0, 1, 3, 6] -2.174291894262031e-05
[0, 1, 6, 7] 0.005832226553862655
[0, 2, 3, 5